In [1]:
import cv2
import numpy as nump
import math

In [10]:
capture = cv2.VideoCapture(0)

while capture.isOpened():
    
    # Aqui se captura el marco
    ret, frame = capture.read()
    # Ventana inferior
    cv2.rectangle(frame,(100,100),(300,300),(0,255,0),0)
    crop_image = frame[100:300, 100:300]
    # Filtro blur
    blur = cv2.GaussianBlur(crop_image, (3,3), 0)
    
    
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    
    
    mask2 = cv2.inRange(hsv, nump.array([2,0,0]), nump.array([20,255,255]))
    
    # Kernel para transformaciones de forma
    kernel = nump.ones((5,5))
    # Transformaciones de forma para eliminar ruido de fondo
    dilation = cv2.dilate(mask2, kernel, iterations = 1)
    erosion = cv2.erode(dilation, kernel, iterations = 1)    
       
    # Filtro Guasiano
    filtered = cv2.GaussianBlur(erosion, (3,3), 0)
    ret,thresh = cv2.threshold(filtered, 127, 255, 0)
    
    # Show threshold image
    cv2.imshow("Thresholded", thresh)

    # Find contours
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    
    try:
        contour = max(contours, key = lambda x: cv2.contourArea(x))
        
        x,y,w,h = cv2.boundingRect(contour)
        cv2.rectangle(crop_image,(x,y),(x+w,y+h),(0,0,255),0)
        
        hull = cv2.convexHull(contour)
        
        drawing = nump.zeros(crop_image.shape, nump.uint8)
        cv2.drawContours(drawing,[contour],-1,(0,255,0),0)
        cv2.drawContours(drawing,[hull],-1,(0,0,255),0)
        
        hull = cv2.convexHull(contour, returnPoints=False)
        defects = cv2.convexityDefects(contour,hull)
        
        count_defects = 0
        
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
            start = tuple(contour[s][0])
            end = tuple(contour[e][0])
            far = tuple(contour[f][0])

            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
            angle = (math.acos((b**2 + c**2 - a**2)/(2*b*c))*180)/3.14
            

            if angle <= 90:
                count_defects += 1
                cv2.circle(crop_image,far,1,[0,0,255],-1)

            cv2.line(crop_image,start,end,[0,255,0],2)

        # Print number of fingers
        if count_defects == 0:
            cv2.putText(frame,"Seguir trabajando", (50,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, 2)
        elif count_defects == 1:
            cv2.putText(frame,"Paz", (50,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, 2)
        elif count_defects == 2:
            cv2.putText(frame, "Agua", (5,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, 2)
        elif count_defects == 3:
            cv2.putText(frame,"Violencia domestica", (50,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, 2)
        elif count_defects == 4:
            cv2.putText(frame,"Nadie se mueva", (50,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, 2)
        else:
            pass
    except:
        pass

    
    cv2.imshow("Gesture", frame)
    all_image = nump.hstack((drawing, crop_image))
    cv2.imshow('Contours', all_image)
      
    # al precionar'a' se para la camara
    if cv2.waitKey(1) == ord('a'):
        break

capture.release()
cv2.destroyAllWindows()

In [ ]:
#no